## Association analysis with the Apriori algorithm

- Pros: Easy to code up
- Cons: May be slow on large datasets
- Works with: Numeric values, nominal values

### Implement Apriori

In [1]:
# Apriori algorithm helper functions
# For each transaction in tran the dataset:
# For each candidate itemset, can:
#     Check to see if can is a subset of tran
#     If so increment the count of can
# For each candidate itemset:
#     If the support meets the minimum, keep this item
# Return list of frequent itemsets
def createC1(dataSet):
    C1 = []
    for transaction in dataSet:
        for item in transaction:
            if not [item] in C1:
                C1.append([item])
    C1.sort()
    # Create a frozenset of each item in C1
    return list(map(frozenset, C1))


def scanD(D, Ck, minSupport):
    ssCnt = {}
    for tid in D:
        for can in Ck:
            if can.issubset(tid):
                if can not in ssCnt:
                    ssCnt[can] = 1
                else:
                    ssCnt[can] += 1
    numItems = len(D)
    retList = []
    supportData = {}
    # Calculate support for every itemset
    for key in ssCnt:
        support = ssCnt[key] / numItems
        if support >= minSupport:
            retList.insert(0, key)
        supportData[key] = support
    return retList, supportData

In [2]:
# The Apriori algorithm
# While the number of items in the set is greater than 0:
#     Create a list of candidate itemsets of length k
#     Scan the dataset to see if each itemset is frequent
#     Keep frequent itemsets to create itemsets of length k+1
def aprioriGen(Lk, k):
    retList = []
    lenLk = len(Lk)
    for i in range(lenLk):
        for j in range(i + 1, lenLk):
            # Join sets if first k-2 items are equal
            L1 = list(Lk[i])[:k - 2]
            L2 = list(Lk[j])[:k - 2]
            L1.sort()
            L2.sort()
            if L1 == L2:
                retList.append(Lk[i] | Lk[j])
    return retList


def apriori(dataSet, minSupport=0.5):
    C1 = createC1(dataSet)
    D = list(map(set, dataSet))
    L1, supportData = scanD(D, C1, minSupport)
    L = [L1]
    k = 2
    while len(L[k - 2]) > 0:
        Ck = aprioriGen(L[k - 2], k)
        # Scan data set to get Lk from Ck
        Lk, supK = scanD(D, Ck, minSupport)
        supportData.update(supK)
        L.append(Lk)
        k += 1
    return L, supportData

In [3]:
# Association rule-generation functions
def generateRules(L, supportData, minConf=0.7):
    bigRuleList = []
    # Get only sets with two or more items
    for i in range(1, len(L)):
        for freqSet in L[i]:
            H1 = [frozenset([item]) for item in freqSet]
            if i > 1:
                rulesFromConseq(freqSet, H1, supportData, bigRuleList, minConf)
            else:
                calcConf(freqSet, H1, supportData, bigRuleList, minConf)
    return bigRuleList


def calcConf(freqSet, H, supportData, brl, minConf=0.7):
    prunedH = []
    for conseq in H:
        conf = supportData[freqSet] / supportData[freqSet - conseq]
        if conf >= minConf:
            print(freqSet - conseq, '-->', conseq, 'conf:', conf)
            brl.append((freqSet - conseq, conseq, conf))
            prunedH.append(conseq)
    return prunedH


def rulesFromConseq(freqSet, H, supportData, brl, minConf=0.7):
    m = len(H[0])
    # Try further merging
    if len(freqSet) > m + 1:
        # Create Hm+1 new candidates
        Hmp1 = aprioriGen(H, m + 1)
        Hmp1 = calcConf(freqSet, Hmp1, supportData, brl, minConf)
        if len(Hmp1) > 1:
            rulesFromConseq(freqSet, Hmp1, supportData, brl, minConf)

### Experiment 1: Toy dataset

In [4]:
def loadDataSet():
    return [[1, 3, 4], [2, 3, 5], [1, 2, 3, 5], [2, 5]]

In [5]:
dataSet = loadDataSet()

In [6]:
L, suppData = apriori(dataSet)

In [7]:
L

[[frozenset({5}), frozenset({2}), frozenset({3}), frozenset({1})],
 [frozenset({2, 3}), frozenset({3, 5}), frozenset({2, 5}), frozenset({1, 3})],
 [frozenset({2, 3, 5})],
 []]

In [8]:
L, suppData = apriori(dataSet, minSupport=0.7)

In [9]:
L

[[frozenset({5}), frozenset({2}), frozenset({3})], [frozenset({2, 5})], []]

In [10]:
L, suppData = apriori(dataSet)

In [11]:
rules = generateRules(L, suppData)

frozenset({5}) --> frozenset({2}) conf: 1.0
frozenset({2}) --> frozenset({5}) conf: 1.0
frozenset({1}) --> frozenset({3}) conf: 1.0


In [12]:
rules = generateRules(L, suppData, minConf=0.5)

frozenset({3}) --> frozenset({2}) conf: 0.6666666666666666
frozenset({2}) --> frozenset({3}) conf: 0.6666666666666666
frozenset({5}) --> frozenset({3}) conf: 0.6666666666666666
frozenset({3}) --> frozenset({5}) conf: 0.6666666666666666
frozenset({5}) --> frozenset({2}) conf: 1.0
frozenset({2}) --> frozenset({5}) conf: 1.0
frozenset({3}) --> frozenset({1}) conf: 0.6666666666666666
frozenset({1}) --> frozenset({3}) conf: 1.0
frozenset({5}) --> frozenset({2, 3}) conf: 0.6666666666666666
frozenset({3}) --> frozenset({2, 5}) conf: 0.6666666666666666
frozenset({2}) --> frozenset({3, 5}) conf: 0.6666666666666666


### Experiment 2: Congressional voting dataset

In [13]:
# API no longer free, use cached data provided by the author
input_file = open("meaning20.txt")
itemMeaning = [line.strip() for line in input_file.readlines()]
input_file.close()
input_file = open("bills20DataSet.txt")
lines = [line.strip() for line in input_file.readlines()]
assert lines[0][0] == "("
lines = lines[1:]
input_file.close()
dataSet, cur = [], []
for line in lines:
    if line[0] == "(":
        dataSet.append(cur)
        cur = []
    elif line[0] == "I":
        cur.append(int(line[1:]))
    elif line[:2] == "aI":
        cur.append(int(line[2:]))
    else:
        raise ValueError
dataSet.append(cur)

In [14]:
L, suppData = apriori(dataSet, minSupport=0.3)

In [15]:
rules = generateRules(L, suppData, minConf=0.99)

frozenset({3}) --> frozenset({0}) conf: 0.9956140350877194
frozenset({3}) --> frozenset({9}) conf: 1.0
frozenset({3}) --> frozenset({0, 9}) conf: 0.9956140350877194
frozenset({3, 7}) --> frozenset({0, 9}) conf: 0.9954954954954954
frozenset({3, 23}) --> frozenset({0, 9}) conf: 0.9954545454545454
frozenset({25, 3}) --> frozenset({0, 9}) conf: 0.9953488372093025
frozenset({26, 3}) --> frozenset({0, 9}) conf: 1.0
frozenset({3, 4}) --> frozenset({0, 9}) conf: 0.9953488372093025
frozenset({26, 3, 4}) --> frozenset({0, 9}) conf: 1.0
frozenset({25, 3, 4}) --> frozenset({0, 9}) conf: 0.9950495049504952
frozenset({3, 4, 23}) --> frozenset({0, 9}) conf: 0.9951923076923077
frozenset({25, 26, 3}) --> frozenset({0, 9}) conf: 1.0
frozenset({26, 3, 23}) --> frozenset({0, 9}) conf: 1.0
frozenset({25, 3, 23}) --> frozenset({0, 9}) conf: 0.9951690821256038
frozenset({26, 3, 7}) --> frozenset({0, 9}) conf: 1.0
frozenset({25, 3, 7}) --> frozenset({0, 9}) conf: 0.9952153110047847
frozenset({3, 23, 7}) --> f

In [16]:
def pntRules(ruleList, itemMeaning):
    for ruleTup in ruleList:
        for item in ruleTup[0]:
            print(itemMeaning[item])
        print("           -------->")
        for item in ruleTup[1]:
            print(itemMeaning[item])
        print("confidence: %f" % ruleTup[2])
        print()

In [17]:
pntRules(rules[:3], itemMeaning)

Prohibiting Federal Funding of National Public Radio -- Yea
           -------->
Republican
confidence: 0.995614

Prohibiting Federal Funding of National Public Radio -- Yea
           -------->
Repealing the Health Care Bill -- Yea
confidence: 1.000000

Prohibiting Federal Funding of National Public Radio -- Yea
           -------->
Republican
Repealing the Health Care Bill -- Yea
confidence: 0.995614



### Experiment 3: Mushroom dataset

In [18]:
mushDatSet = [line.split() for line in open('mushroom.dat').readlines()]
L, suppData = apriori(mushDatSet, minSupport=0.3)

In [19]:
print_cnt = 0
for item in L[1]:
    if item.intersection('2'):
        print(item)
        print_cnt += 1
        if print_cnt == 5:
            break

frozenset({'2', '28'})
frozenset({'2', '53'})
frozenset({'2', '23'})
frozenset({'2', '34'})
frozenset({'2', '36'})


In [20]:
print_cnt = 0
for item in L[3]:
    if item.intersection('2'):
        print(item)
        print_cnt += 1
        if print_cnt == 5:
            break

frozenset({'34', '28', '63', '2'})
frozenset({'86', '34', '28', '2'})
frozenset({'34', '28', '2', '59'})
frozenset({'28', '63', '2', '59'})
frozenset({'86', '28', '2', '59'})
